In [2]:
import pickle
import itertools
from ml_model.ml import uniprot_gene_symbol,ensembol_gene_symbol
import pandas as pd

In [3]:
data_saved = pickle.load(open('../results/result.out', 'rb'))

In [5]:
drugs  = ['Bortezomib',
          'Carfilzomib',
          'Ixazomib',
          'Oprozomib']

In [15]:
def get_common_gene_drugs(data_saved):
    common_gene_dict = {}
    common_gene_dict_symbol = {}
    for p_n_gene in ['final_pcm','final_ncm']:
        genes = set.intersection(*[set(data_saved[p_n_gene][d][0]) for d in drugs])
        genes_symbol = uniprot_gene_symbol(list(genes))
        genes_merged = set.intersection(*[set(itertools.chain.from_iterable(data_saved[p_n_gene][d])) for d in drugs])
        genes_merged_symbol = uniprot_gene_symbol(list(genes_merged))
        common_gene_dict[p_n_gene] = {'all_module':genes_merged,'first_module':genes}
        common_gene_dict_symbol[p_n_gene] = {'all_module':genes_merged_symbol,'first_module':genes_symbol}
    return common_gene_dict, common_gene_dict_symbol

In [16]:
common_gene_dict, common_gene_dict_symbol = get_common_gene_drugs(data_saved)
common = pd.DataFrame(common_gene_dict_symbol)
common.to_csv('../results/common.csv')

querying 1-20...done.
Finished.
querying 1-39...done.
Finished.
querying 1-17...done.
Finished.
3 input query terms found no hit:
	['Q9GIY3', 'P04229', 'Q29974']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-30...done.
Finished.
3 input query terms found no hit:
	['P04229', 'Q29974', 'Q9GIY3']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [11]:
def get_single_drug_fdr():
    classic = pd.read_csv('../results/mm106_data_cor_pvalue_0.05.csv', sep=';')
    classic = classic[round(classic['adjust_pvalue'], 2) <0.1]
    classic_p = classic[round(classic['coefficient'], 2) >0]
    classic_n = classic[round(classic['coefficient'], 2) <0]
    drug_classic_dict_p = dict(classic_p.groupby('drug')['Gene'].apply(list))
    drug_classic_dict_n = dict(classic_n.groupby('drug')['Gene'].apply(list))
    classic_gene = {}
    for d in drugs:
        classic_gene[d] = {'final_pcm_symbol':drug_classic_dict_p[d],
                           'final_ncm_symbol':drug_classic_dict_n[d]}
    return classic_gene

classic_gene_dict = get_single_drug_fdr()

In [13]:
def get_group_fdr_gene():
    group_gene = pd.read_csv('../DEGs/bortezomib_count_0.05.csv',index_col=0)
    group_gene_up = group_gene.index[group_gene['log2FoldChange']>0].values
    group_gene_up_symbol = list(ensembol_gene_symbol(group_gene_up).values())
    group_gene_down = group_gene.index[group_gene['log2FoldChange']<0].values
    group_gene_down_symbol = list(ensembol_gene_symbol(group_gene_down).values())
    return {'Bortezomib':{'final_ncm_symbol':group_gene_up_symbol,
                           'final_pcm_symbol':group_gene_down_symbol}}
group_fdr_gene_dict = get_group_fdr_gene()

In [8]:
def get_common_gene_methods(drugs, data_saved, group_fdr_gene_dict, classic_gene_dict ):
    common_gene_dict = {}
    for d in drugs:
        common_gene_dict_drug = {}
        for p_n_gene in ['final_pcm_symbol','final_ncm_symbol']:
            a = sorted(set(data_saved[p_n_gene][d][0]))
            b = sorted(set(group_fdr_gene_dict[d][p_n_gene]))
            c = sorted(set(classic_gene_dict[d][p_n_gene]))

            common_genes = set.intersection(set(a),
                                            set(b),
                                            set(c))


            a_merged = sorted(set(itertools.chain.from_iterable(data_saved[p_n_gene][d])))

            common_genes_merged = set.intersection(set(a_merged),
                                                    set(b),
                                                   set(c))
            common_gene_dict_drug[p_n_gene] = {'all_module':common_genes_merged,'first_module':common_genes}
        common_gene_dict[d] =  common_gene_dict_drug
    return common_gene_dict

common_gene_dict = get_common_gene_methods(['Bortezomib'], data_saved, group_fdr_gene_dict, classic_gene_dict)

querying 1-169...done.
Finished.
querying 1-218...done.
Finished.
querying 1-110...done.
Finished.
1 input query terms found dup hits:
	[('P0DMV8', 2)]
3 input query terms found no hit:
	['P04229', 'Q9GIY3', 'Q29974']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-170...done.
Finished.
1 input query terms found dup hits:
	[('P0DMV8', 2)]
3 input query terms found no hit:
	['P04229', 'Q9GIY3', 'Q29974']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-161...done.
Finished.
querying 1-211...done.
Finished.
querying 1-69...done.
Finished.
8 input query terms found no hit:
	['P04229', 'Q31612', 'Q31610', 'Q9GIY3', 'P30486', 'Q29974', 'P30480', 'Q29836']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-98...done.
Finished.
8 input query terms found no hit:
	['Q31612', 'P04229', 'Q31610', 'Q9GIY3', 'P30486', 'Q29974', 'Q29836', 'P30480']
Pass "returnall=

In [ ]:
with(open('../results/common_genes_methods-dict')) as handle:
    pickle.dump(common_gene_dict, handle)

In [12]:
#common_gene_dict = pickle.load(open('../results/common_genes_methods_dict','rb'))
#Writing final modules
with open('../results/final_modules_common_fdr_0.05.txt','w') as o:
    for drug,gene_dict in common_gene_dict.items():
        for module_type, module_id_dict in gene_dict.items():
            for module_id, genes in  module_id_dict.items():
                o.write('{} \t {} \t {} \t'.format(drug,module_type, module_id) +'\t'.join(genes)+'\n')
